In [18]:
import util;
import os;
import numpy as np;
import visualize;
dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000';

def parseAnnoStr(annos):
    annos=[int(num) for num in annos];
    annos=np.array(annos);
    annos=np.reshape(annos,(len(annos)/2,2));
    return annos;

def readGTFile(file_curr):
    lines=util.readLinesFromFile(file_curr);
    im_paths=[];
    ims_size=[];
    annos_all=[];
    for line_curr in lines:
        line_split=line_curr.rsplit(None,14);
        im_paths.append(line_split[0]);
        
        im_size=line_split[1:1+4];
        im_size=[int(num) for num in im_size];
        im_size=[im_size[2]-im_size[0],im_size[3]-im_size[1]];
        ims_size.append(im_size);
        
        annos=line_split[1+4:];
        annos=parseAnnoStr(annos);
        annos_all.append(annos);
        
    return im_paths,ims_size,annos_all;
        
def readPredFile(pred_file):
    lines=util.readLinesFromFile(pred_file);
    annos_all=[];
    for line_curr in lines:
        annos_str=line_curr.split();
        annos=parseAnnoStr(annos_str);
        annos_all.append(annos);
    return annos_all;

def getDiffs(annos_gt,annos_pred):
    diffs_all=[];
    for anno_gt,anno_pred in zip(annos_gt,annos_pred):
        diffs=np.power(anno_gt-anno_pred,2);
        diffs=np.sum(diffs,1);
        diffs=np.power(diffs,0.5);
        diffs_all.append(diffs);
    return diffs_all;

def getErrorPercentageImSize(im_sizes,diffs_all):
    errors_all=[];
    for im_size,diffs in zip(im_sizes,diffs_all):
        assert (im_size[0]==im_size[1]);
        errors=-1*np.ones(diffs.shape);
        errors[diffs>0]=diffs[diffs>0]/im_size[0];
        errors_all.append(errors);
    return errors_all;

def us_getFilePres(gt_file,out_dir_us,post_us,num_iter,batch_us):
    files_gt=[];
    files_pred=[];
    im_paths=util.readLinesFromFile(gt_file);
    im_paths=[im_path[:im_path.index(' ')] for im_path in im_paths];
    num_gt=len(im_paths);
    count=0;
    for batch_num in range(num_iter):
        for im_num in range(batch_us):
            file_pre=str(batch_num+1)+'_'+str(im_num+1);
            file_gt=file_pre+post_us[0];
            file_pred=file_pre+post_us[1];
            files_gt.append(os.path.join(out_dir_us,file_gt));
            files_pred.append(os.path.join(out_dir_us,file_pred));
#             print file_pred;
#             print file_gt
            
    files_gt=files_gt[:num_gt];
    files_pred=files_pred[:num_gt];
    return im_paths,files_gt,files_pred;

def us_getPredGTPairs(gt_pt_files,pred_pt_files):
    annos_gt=[];
    annos_pred=[];
    for gt_file,pred_file in zip(gt_pt_files,pred_pt_files):
        gt_pts=np.load(gt_file);
        pred_pts=np.load(pred_file);
        bin_keep=gt_pts[:,2]>0
        gt_pts=gt_pts[bin_keep,:2];
        pred_pts=pred_pts[bin_keep,:2];
        annos_gt.append(gt_pts);
        annos_pred.append(pred_pts);
        
    return annos_gt,annos_pred;

def us_getDiffs(gt_pt_files,pred_pt_files):
    diffs_all=[];
    for gt_file,pred_file in zip(gt_pt_files,pred_pt_files):
        gt_pts=np.load(gt_file);
        pred_pts=np.load(pred_file);
        bin_keep=gt_pts[:,2]>0
        diffs_curr=-1*np.ones((gt_pts.shape[0],));
        gt_pts=gt_pts[bin_keep,:2];
        pred_pts=pred_pts[bin_keep,:2];
        diffs=np.power(gt_pts-pred_pts,2);
        diffs=np.sum(diffs,1);
        diffs=np.power(diffs,0.5);
        diffs_curr[bin_keep]=diffs;
        diffs_all.append(diffs_curr);
    return diffs_all;


def us_getErrorsAll(gt_file,out_dir_us,post_us,num_iter,batch_size):
    im_paths,gt_pt_files,pred_pt_files=us_getFilePres(gt_file,out_dir_us,post_us,num_iter,batch_size);
    
#     annos_gt,annos_pred=us_getPredGTPairs(gt_pt_files,pred_pt_files);
#     diffs_all=getDiffs(annos_gt,annos_pred);
    diffs_all=us_getDiffs(gt_pt_files,pred_pt_files);
    print diffs_all[0][0];
    im_sizes=[[2.0,2.0]]*len(diffs_all)
    errors_all=getErrorPercentageImSize(im_sizes,diffs_all);
    return errors_all;

def them_getErrorsAll(gt_file,pred_file):
    im_paths,im_sizes,annos_gt=readGTFile(gt_file);
    annos_pred=readPredFile(pred_file);
    diffs_all=getDiffs(annos_gt,annos_pred);
    errors_all=getErrorPercentageImSize(im_sizes,diffs_all);
    return errors_all;

def plotComparisonCurve(errors_all,out_file,labels):
    vals=[];
    for err in errors_all:
        err=np.array(err);
#         avg=np.mean(err,1);
        bin_keep=err>=0;
        err[err<0]=0;
        div=np.sum(bin_keep,1);
        sum_val=np.sum(err,1).astype(np.float);
        avg=sum_val/div;
        
        avg=np.sort(avg);
        vals.append(avg);
        
    xAndYs=[(range(len(val_curr)),val_curr) for val_curr in vals];
    xlabel='Sorted Image Number';
    ylabel='BBox Normalized Error';
    visualize.plotSimple(xAndYs,out_file,xlabel=xlabel,ylabel=ylabel,legend_entries=labels);
    
#     plotSimple(xAndYs,out_file,title='',xlabel='',ylabel='',legend_entries=None,loc=0,outside=False,logscale=False)
        
def plotComparisonKpAvgError(errors_all,out_file,ticks,labels,xlabel=None,ylabel=None,colors=None,ylim=None,title=''):
    vals={};
    for err,label_curr in zip(errors_all,labels):
        err=np.array(err);
        bin_keep=err>=0;
        err[err<0]=0;
        div=np.sum(bin_keep,0);
        sum_val=np.sum(err,0).astype(np.float);
        avg=sum_val/div;
#         avg=np.mean(err,0);
        
        
        vals[label_curr]=avg;

    if colors is None:
        colors=['b','g'];
    if xlabel is None:
        xlabel='Keypoint';
        
    if ylabel is None:
        ylabel='BBox Normalized Error';
        
    visualize.plotGroupBar(out_file,vals,ticks,labels,colors,xlabel=xlabel,ylabel=ylabel,\
                           width=1.0/len(vals),ylim=ylim,title=title);
#     ,title='',width=0.25,ylim=None)


def getErrRates(err,thresh=0.1):
#     vals=[];
#     total_errRate=[];
#     for err in zip(errors_all):
    err=np.array(err);
    print type(err);
    sum_errs=np.sum(err>thresh,0).astype(np.float);
    print 'sum_errs',sum_errs;
    total_errs=np.sum(err>=0,0);
#     print total_errs.shape;
#     print sum_errs.shape;
    err_rate=sum_errs/total_errs*100.0;
    print err_rate;
    sum_errs_tot=np.sum(sum_errs);
    total_errs_tot=np.sum(total_errs);
    err_rate_tot=sum_errs_tot/total_errs_tot*100.0;
#         total_errRate.append(err_rate_tot);
#         vals.append(err_rate);
    return err_rate,err_rate_tot;

def plotComparisonKpError(errors_all,out_file,ticks,labels,xlabel=None,ylabel=None,colors=None,thresh=0.1,\
                          title='',ylim=None):
    vals={};
    err_check=np.array(errors_all);
    err_rates_all=[];
    for err in errors_all:
#         print err_check.shape;
        err_rate,err_rate_tot=getErrRates(err);
        print err_rate;
        if len(labels)==len(err_rate)+1:
            err_rate.append(err_rate_tot);
        err_rates_all.append(err_rate);
    err_rates_all=np.array(err_rates_all);
    print err_rates_all;
#     assert len(labels)==err_rates_all.shape[1];
    for idx_label_curr,label_curr in enumerate(labels):
        vals[label_curr]=err_rates_all[:,idx_label_curr];
    print vals;
        
#     for err_rate_curr,label_curr in zip(err_rate,labels):
#         vals[label_curr]=err_rate_curr;
#     for err,label_curr in zip(errors_all,labels):
#         err=np.array(err);
#         sum_errs=np.sum(err>thresh,0).astype(np.float);
#         total_errs=np.sum(err>=0,0);
# #         print total_errs;
# #         print sum_errs;
#         err_rate=sum_errs/total_errs*100.0;
# #         print err_rate;
#         vals[label_curr]=list(err_rate);
#         vals['ALL']=[]
    
#     vals[labels[0]].append(11.577);
#     vals[labels[1]].append(13.75);

    if colors is None:
        colors=['b','g'];
        
    if xlabel is None:
        xlabel='Keypoint';
        
    if ylabel is None:
        ylabel='Failure Rate %';
 
        
    visualize.plotGroupBar(out_file,vals,ticks,labels,colors,xlabel=xlabel,ylabel=ylabel,\
                           width=1.0/len(vals),title=title,ylim=ylim);

#     pass;

def writeJustTestScript(out_file_sh,val_data_path,iterations,batch_size,model_out_tups,face):    
    file_th='/home/maheenrashid/Downloads/horses/torch/justTest.th';
    commands_all=[];
    for model_path_curr,out_dir_curr in model_out_tups:
        command_curr=['th',file_th];
        command_curr=command_curr+['-val_data_path',val_data_path];
        command_curr=command_curr+['-iterations',str(iterations)];
        command_curr=command_curr+['-batchSize',str(batch_size)];
        command_curr=command_curr+['-full_model_path',model_path_curr];
        command_curr=command_curr+['-outDirTest',out_dir_curr];
        if face:
            command_curr=command_curr+['-face'];
        command_curr=' '.join(command_curr);
#         print command_curr;
        commands_all.append(command_curr);
    
    util.writeFile(out_file_sh,commands_all);
    print len(commands_all);
    print out_file_sh;


def sheepPeopleComparisonScript():
    dir_sheep_results=os.path.join(dir_server,'horse_project/sheep_baseline_results');
    dir_input_data='/home/SSD3/maheen-data/horse_project/files_for_sheepCode'
    util.mkdir(dir_sheep_results);
    
    us_test=['sheep_test_us_sheep_minloss.txt','horse_test_us_horse_minloss.txt'];
    us_test=[os.path.join(dir_input_data,file_curr) for file_curr in us_test]
    out_us=[os.path.join(dir_sheep_results,'sheep_us/test_images'),\
    os.path.join(dir_sheep_results,'horse_us/test_images')];
    batch_size=50;
    num_iter=2;
    post_us=['_gt_pts.npy','_pred_pts.npy']
    im_size=2;
    
#     them_test=['sheep_test.txt','horse_test.txt'];
    them_test=['sheep_test_new.txt','horse_test_new.txt'];
    them_test=[os.path.join(dir_input_data,file_curr) for file_curr in them_test];
    out_them=[file_curr[:file_curr.rindex('.')]+'_TIF_result.txt' for file_curr in them_test];
    
    out_file_curve_pre=os.path.join(dir_sheep_results,'curve_comparison_new');
#                                 .png');
    out_file_curve_3_pre=os.path.join(dir_sheep_results,'curve_comparison_just3_new');
#                                   .png');
    
    out_file_kp_avg_pre=os.path.join(dir_sheep_results,'avg_kp_comparison_new');
    out_file_kp_err_pre=os.path.join(dir_sheep_results,'failure_kp_comparison_new')
    
    labels=['Ours','TIF'];
#              Sheep'],['Ours Horse','TIF Horse']];
    out_file_post_tags=[ 'Sheep','Horse']
    ticks=['LE','RE','N','LM','RM'];
    
    errors_all=[];
    
    for gt_file_them,pred_file_them,gt_file_us,out_dir_us,out_file_post_tag_curr in \
            zip(them_test,out_them,us_test,out_us,out_file_post_tags):
        us_errors_all=us_getErrorsAll(gt_file_us,out_dir_us,post_us,num_iter,batch_size)
        them_errors_all=them_getErrorsAll(gt_file_them,pred_file_them);

        errors_all=[us_errors_all,them_errors_all];
        
        labels_curr=[label_curr+'\n'+out_file_post_tag_curr for label_curr in labels];
        if out_file_post_tag_curr=='Sheep':
            labels_curr[0]=labels_curr[0]+' ';
        
        out_file_curve=out_file_curve_pre+'_'+out_file_post_tag_curr+'.pdf';
        plotComparisonCurve(errors_all,out_file_curve,labels_curr);
        out_file_curve=out_file_curve_pre+'_'+out_file_post_tag_curr+'.png';
        plotComparisonCurve(errors_all,out_file_curve,labels_curr);
        print out_file_curve.replace(dir_server,click_str);
        
        errors_3=[np.array(err)[:,:2] for err in errors_all];
#         labels_curr=labels;
#         title=out_file_post_tag_curr;
#         out_file_curve_3=out_file_curve_3_pre+'_'+out_file_post_tag_curr+'.pdf';
#         plotComparisonCurve(errors_3,out_file_curve_3,labels_curr);
#         out_file_curve_3=out_file_curve_3_pre+'_'+out_file_post_tag_curr+'.png';
#         plotComparisonCurve(errors_3,out_file_curve_3,labels_curr);
#         print out_file_curve_3.replace(dir_server,click_str);
        ylim=None
        title=''
        out_file_kp_avg=out_file_kp_avg_pre+'_'+out_file_post_tag_curr+'.pdf';
        plotComparisonKpAvgError(errors_all,out_file_kp_avg,ticks,labels_curr,ylim=ylim);
        out_file_kp_avg=out_file_kp_avg_pre+'_'+out_file_post_tag_curr+'.png';
        plotComparisonKpAvgError(errors_all,out_file_kp_avg,ticks,labels_curr,title=title,ylim=ylim);

        print out_file_kp_avg.replace(dir_server,click_str);
        ylim=None;
        out_file_kp_err=out_file_kp_err_pre+'_'+out_file_post_tag_curr+'.pdf';
        plotComparisonKpError(errors_all,out_file_kp_err,ticks,labels_curr,ylim=ylim);
        out_file_kp_err=out_file_kp_err_pre+'_'+out_file_post_tag_curr+'.png';
        plotComparisonKpError(errors_all,out_file_kp_err,ticks,labels_curr,title=title,ylim=ylim);

        print out_file_kp_err.replace(dir_server,click_str);



def ourComparisonScript():        
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    dir_figures='/home/SSD3/maheen-data/horse_project/cvpr_figs';
    iterations=2;
    batch_size=100;
    post_us=['_gt_pts.npy','_pred_pts.npy']
    ticks=['LE','RE','N','LM','RM'];
    
    out_dir_us=os.path.join(dir_figures,'full_system_2loss_tps/test_images');
    out_dir_us_affine=os.path.join(dir_figures,'full_system_2loss_affine/test_images');
    out_dir_bl_ft=os.path.join(dir_figures,'baseline_finetune/test_images');
    out_dir_bl_tps=os.path.join(dir_figures,'baseline_tps/test_images');
    out_dir_bl_affine=os.path.join(dir_figures,'baseline_affine/test_images');
    out_dir_random=os.path.join(dir_figures,'random_neighbors');
    
    out_dir_us_sheep=os.path.join(dir_figures,'full_system_2loss_tps_sheep/test_images');
    out_dir_bl_ft_sheep=os.path.join(dir_figures,'baseline_finetune_sheep/test_images');
    out_dir_bl_tps_sheep=os.path.join(dir_figures,'baseline_tps_sheep/test_images');


#     dirs_test_all=[out_dir_us,out_dir_bl_ft,out_dir_bl_tps];
#     labels=['Ours','BL Finetune','BL TPS'];
#     out_file_curve=os.path.join(dir_figures,'curve_comparison_us_ft_tps.pdf');
#     out_file_kp_err=os.path.join(dir_figures,'failure_kp_comparison_us_ft_tps.pdf');
#     out_file_kp_avg=os.path.join(dir_figures,'avg_kp_comparison_us_ft_tps.pdf');
#     ylim=[0,0.09];
#     colors=['b','g','r'];
    
#     dirs_test_all=[out_dir_us,out_dir_us_affine,out_dir_bl_ft,out_dir_bl_tps,out_dir_bl_affine];
#     labels=['Ours TPS','Ours Affine','BL Finetune','BL TPS','BL Affine'];
#     out_file_curve=os.path.join(dir_figures,'curve_comparison_us_ft_tps_affine.pdf');
#     out_file_kp_err=os.path.join(dir_figures,'failure_kp_comparison_us_ft_tps_affine.pdf');
#     out_file_kp_avg=os.path.join(dir_figures,'avg_kp_comparison_us_ft_tps_affine.pdf');
#     ylim=[0,0.11]
#     colors=['b','c','g','r','y'];
    
#     iterations=2;
#     batch_size=50;

#     dirs_test_all=[out_dir_us_sheep,out_dir_bl_ft_sheep,out_dir_bl_tps_sheep];
#     labels=['Ours','BL Finetune','BL TPS'];
#     out_file_curve=os.path.join(dir_figures,'curve_comparison_us_ft_tps_sheep.pdf');
#     out_file_kp_err=os.path.join(dir_figures,'failure_kp_comparison_us_ft_tps_sheep.pdf');
#     out_file_kp_avg=os.path.join(dir_figures,'avg_kp_comparison_us_ft_tps_sheep.pdf');
#     ylim=[0,0.06];
#     colors=['b','g','r'];
    
#     dirs_test_all=[out_dir_us_sheep,out_dir_bl_ft_sheep]
#     labels=['Ours','BL Finetune'];
#     out_file_curve=os.path.join(dir_figures,'curve_comparison_us_ft_sheep.pdf');
#     out_file_kp_err=os.path.join(dir_figures,'failure_kp_comparison_us_ft_sheep.pdf');
#     out_file_kp_avg=os.path.join(dir_figures,'avg_kp_comparison_us_ft_sheep.pdf');
#     ylim=None;
#     colors=['b','g'];

    dirs_test_all=[out_dir_us,out_dir_random]
    labels=['Ours TPS','Ours Random'];
    out_file_curve=os.path.join(dir_figures,'curve_comparison_us_ft_random.pdf');
    out_file_kp_err=os.path.join(dir_figures,'failure_kp_comparison_us_ft_random.pdf');
    out_file_kp_avg=os.path.join(dir_figures,'avg_kp_comparison_us_ft_random.pdf');
    ylim=[0,23];
    colors=['b','g'];
    
    errors_all=[];
    for out_dir_test in dirs_test_all:
        im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,out_dir_test,post_us,iterations,batch_size);
        errors_curr=us_getErrorsAll(val_data_path,out_dir_test,post_us,iterations,batch_size);
        failures,failures_kp=getErrRates(errors_curr,0.1)
        print failures_kp;
        errors_all.append(errors_curr)
    ticks.append('ALL');
#     errors_all.append()
#     plotComparisonCurve(errors_all,out_file_curve,labels);
#     print out_file_curve.replace(dir_server,click_str);

    plotComparisonKpError(errors_all,out_file_kp_err,ticks,labels,colors=colors,ylim=ylim);
    print out_file_kp_err.replace(dir_server,click_str);
    
#     plotComparisonKpAvgError(errors_all,out_file_kp_avg,ticks,labels,colors=colors,ylim=ylim);
#     print out_file_kp_avg.replace(dir_server,click_str);



sheepPeopleComparisonScript()
# ourComparisonScript()

0.0484243793964
http://vision1.idav.ucdavis.edu:1000/horse_project/sheep_baseline_results/curve_comparison_new_Sheep.png
http://vision1.idav.ucdavis.edu:1000/horse_project/sheep_baseline_results/avg_kp_comparison_new_Sheep.png
0.1
[ 3.  3.  6.  3.  4.]
[ 3.  3.  6.  3.  4.]
0.1
[ 18.          15.46391753  14.14141414  14.          12.        ]
[ 18.          15.46391753  14.14141414  14.          12.        ]
[[  3.           3.           6.           3.           4.        ]
 [ 18.          15.46391753  14.14141414  14.          12.        ]]
{'Ours\nSheep ': array([  3.,  18.]), 'TIF\nSheep': array([  3.        ,  15.46391753])}


ValueError: incompatible sizes: argument 'height' must be length 5 or scalar

0.0484243793964
[array([ 0.02421219,  0.03177905,  0.02777392,  0.03975768,  0.06588076]), array([ 0.03418528,  0.01699448,  0.01905751,  0.03918555,  0.1132861 ]), array([ 0.04250287,  0.03739381,  0.00502316,  0.06819926,  0.047026  ]), array([ 0.02614648,  0.02568217,  0.0083395 ,  0.03455059,  0.01990674]), array([ 0.06950265,  0.0867721 ,  0.02202129,  0.07150158,  0.0365107 ]), array([ 0.0140258 ,  0.02006528,  0.03009923,  0.01677536,  0.02361475]), array([ 0.03762876,  0.02711951,  0.02508832,  0.01137514,  0.01745636]), array([ 0.02451698,  0.00702299,  0.01462946,  0.02614925,  0.06217187]), array([ 0.0145288 ,  0.02016901,  0.00638304,  0.01114911,  0.00633233]), array([ 0.04569243,  0.03449682,  0.01281828,  0.00917226,  0.02033965]), array([ 0.01917927,  0.03091634,  0.00372409,  0.00881756,  0.02805741]), array([ 0.01322089,  0.00298696,  0.02439833,  0.02526191,  0.02563465]), array([ 0.02025552,  0.01207801,  0.02684091,  0.00436437,  0.02894531]), array([ 0.02498696,  

In [ ]:

def writeTestScript():
#     dir_figures='/home/SSD3/maheen-data/horse_project/cvpr_figs';
    dir_figures='/home/SSD3/maheen-data/horse_project/cvpr_figs_noDuplicates';
    util.mkdir(dir_figures);
    out_file_sh=os.path.join(dir_figures,'commands_test.sh');
    file_th='/home/maheenrashid/Downloads/horses/torch/justTest.th';
#     val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean_noDuplicates.txt';
    iterations=2;
    batch_size=100;
    face=False;
    model_out_tups=[];
    
#     #us_tps
#     full_model_path=os.path.join(dir_server,\
#     'horse_project/full_system_small_data_eye_1e-2_10_100/matches_5_3531_horse_minloss/final/model_all_final.dat');
#     out_dir_test=os.path.join(dir_figures,'full_system_2loss_tps/test_images');
#     model_out_tups.append((full_model_path,out_dir_test))
# #     us_affine
#     full_model_path=os.path.join(dir_server,\
# 'horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_3531_horse_minloss/final/model_all_final.dat');
#     out_dir_test=os.path.join(dir_figures,'full_system_2loss_affine/test_images');
#     model_out_tups.append((full_model_path,out_dir_test))

# #     tps_baseline
#     full_model_path=os.path.join(dir_server,'horse_project/ext_disk/face_baselines_small_data/matches_5_3531_horse_minloss_tps_eye/final/model_all_final.dat');
#     out_dir_test=os.path.join(dir_figures,'baseline_tps/test_images');
#     model_out_tups.append((full_model_path,out_dir_test))
#     #tps_affine
#     full_model_path=os.path.join(dir_server,'horse_project/ext_disk/face_baselines_small_data/matches_5_3531_horse_minloss_affine/final/model_all_final.dat');
#     out_dir_test=os.path.join(dir_figures,'baseline_affine/test_images');
#     model_out_tups.append((full_model_path,out_dir_test))
#    baseline ft
    full_model_path=os.path.join(dir_server,\
'horse_project/ext_disk/face_baselines_small_data/matches_5_3531_horse_minloss/final/model_all_final.dat');
    out_dir_test=os.path.join(dir_figures,'baseline_finetune/test_images');
    face=True;
    model_out_tups.append((full_model_path,out_dir_test))
    
#     full_model_path=os.path.join(dir_server,\
# 'horse_project/vanilla_train_face_big/intermediate/model_all_16762.dat');
#     out_dir_test=os.path.join(dir_figures,'face_network_results/test_images');
#     face=True;
#     model_out_tups.append((full_model_path,out_dir_test))
    
#     val_data_path='/home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_test_allKP_minloss.txt';
#     iterations=2;
#     batch_size=50;
        
#     full_model_path=os.path.join(dir_server,\
# 'horse_project/sheep_models/full_system_2loss/final/model_all_final.dat');
#     out_dir_test=os.path.join(dir_figures,'full_system_2loss_tps_sheep/test_images');
#     face=False;
#     model_out_tups.append((full_model_path,out_dir_test))
    
#     full_model_path=os.path.join(dir_server,\
# 'horse_project/sheep_models/bl_finetune/final/model_all_final.dat');
#     out_dir_test=os.path.join(dir_figures,'baseline_finetune_sheep/test_images');
#     face=True;
#     model_out_tups.append((full_model_path,out_dir_test))

#     full_model_path=os.path.join(dir_server,\
# 'horse_project/sheep_models/bl_tps/final/model_all_final.dat');
#     out_dir_test=os.path.join(dir_figures,'baseline_tps_sheep/test_images');
#     face=False;
#     model_out_tups.append((full_model_path,out_dir_test))

    
    commands_all=[];
    for model_path_curr,out_dir_curr in model_out_tups:
        command_curr=['th',file_th];
        command_curr=command_curr+['-val_data_path',val_data_path];
        command_curr=command_curr+['-iterations',str(iterations)];
        command_curr=command_curr+['-batchSize',str(batch_size)];
        command_curr=command_curr+['-full_model_path',model_path_curr];
        command_curr=command_curr+['-outDirTest',out_dir_curr];
        if face:
            command_curr=command_curr+['-face'];
        command_curr=' '.join(command_curr);
        print command_curr;
        commands_all.append(command_curr);
    
    util.writeFile(out_file_sh,commands_all);
    print out_file_sh;

writeTestScript();

In [ ]:
def makeDirHTML():
    dir_figures='/home/SSD3/maheen-data/horse_project/cvpr_figs';
    dir_sheep_results=os.path.join(dir_server,'horse_project/sheep_baseline_results');

    visualize.writeHTMLForFolder(dir_figures,ext='.png');
    visualize.writeHTMLForFolder(dir_sheep_results,ext='.png');
    print dir_figures.replace(dir_server,click_str);
    print dir_sheep_results.replace(dir_server,click_str);

#     /home/SSD3/maheen-data/horse_project/ext_disk/face_baselines_small_data/matches





In [ ]:
def makeTestScript():
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
#     dir_ablation=os.path.join(dir_server,'horse_project/ablation_studies');
    
#     val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean_noDuplicates.txt';
#     dir_ablation=os.path.join(dir_server,'horse_project/ablation_studies_noDuplicates');
    
#     util.mkdir(dir_ablation);
#     dir_pre='matches_5_';
#     dir_post='_horse_minloss';
    iterations=2;
    batch_size=100;
    
#     dir_out=os.path.join(dir_ablation,'bl_ft');
#     util.mkdir(dir_out);
#     out_file_sh=os.path.join(dir_out,'commands_test.sh')
#     face=True;
    
#     dirs_bl_ft=[];
#     dir_meta=os.path.join(dir_server,'horse_project/ext_disk/face_baselines_small_data');
#     nums=[str(num) for num in [2500]];
# #     500,1000,1500,2000,3000,3531
#     dirs_bl_ft=dirs_bl_ft+[os.path.join(dir_meta,dir_pre+num_curr+dir_post) for num_curr in nums];
#     out_dirs=[os.path.join(dir_out,num_curr,'test_images') for num_curr in nums];
#     model_paths=[os.path.join(dir_curr,'final/model_all_final.dat') for dir_curr in dirs_bl_ft];
#     for model_path in model_paths:
#         assert os.path.exists(model_path);
#     model_out_tups=zip(model_paths,out_dirs);
#     writeJustTestScript(out_file_sh,val_data_path,iterations,batch_size,model_out_tups,face);
    dir_ablation=os.path.join(dir_server,'horse_project/cvpr_figs');
    dir_out=os.path.join(dir_ablation,'random_neighbors');
    util.mkdir(dir_out);
    out_file_sh=os.path.join(dir_out,'commands_test.sh')
    face=False;
    
    dirs_bl_ft=[];
    dir_meta=os.path.join(dir_server,'horse_project/experiment_fake_random_neighbors');
    nums=[str(num) for num in [3531]];
#     500,1000,1500,2000,2500,3000,
#     dirs_bl_ft=dirs_bl_ft+[os.path.join(dir_meta,dir_pre+num_curr+dir_post) for num_curr in nums];
#     out_dirs=[os.path.join(dir_out,num_curr,'test_images') for num_curr in nums];
    out_dirs=[dir_out];
    dirs_bl_ft=[os.path.join(dir_meta,'full_system')]
    model_paths=[os.path.join(dir_curr,'final/model_all_final.dat') for dir_curr in dirs_bl_ft];
    for model_path in model_paths:
        assert os.path.exists(model_path);
    
    model_out_tups=zip(model_paths,out_dirs);
    writeJustTestScript(out_file_sh,val_data_path,iterations,batch_size,model_out_tups,face);

#     dir_out=os.path.join(dir_ablation,'bl_tps');
#     util.mkdir(dir_out);
#     out_file_sh=os.path.join(dir_out,'commands_test.sh')
#     face=False;
#     dir_post='';
#     dirs_bl_ft=[];
#     dir_meta=os.path.join(dir_server,'horse_project/face_baselines_small_data_tps_200_1e-3_100');
#     nums=[str(num) for num in [2000,2500,3000]];
# #     500,1000,1500,
#     dirs_bl_ft=dirs_bl_ft+[os.path.join(dir_meta,dir_pre+num_curr+dir_post) for num_curr in nums];
    
#     dir_meta=os.path.join(dir_server,'horse_project/face_baselines_small_data_tps_200_1e-4_100');
#     nums=[str(num) for num in [500,1000,1500]];
#     dirs_bl_ft=dirs_bl_ft+[os.path.join(dir_meta,dir_pre+num_curr+dir_post) for num_curr in nums];
    
# #     dir_meta=os.path.join(dir_server,'horse_project/full_system_small_data_eye_1e-2_10_100');
# #     nums=[str(num) for num in [3531]];
# #     dir_post='_horse_minloss';
#     dir_curr=os.path.join(dir_server,\
#     'horse_project/ext_disk/face_baselines_small_data/matches_5_3531_horse_minloss_tps_eye');
#     dirs_bl_ft=dirs_bl_ft+[dir_curr]
    
#     nums=[2000,2500,3000,500,1000,1500,3531];
#     nums=[str(num_curr) for num_curr in nums];
#     out_dirs=[os.path.join(dir_out,num_curr,'test_images') for num_curr in nums];
#     model_paths=[os.path.join(dir_curr,'final/model_all_final.dat') for dir_curr in dirs_bl_ft];
    
#     for model_path in model_paths:
#         print model_path
#         assert os.path.exists(model_path);
    
#     model_out_tups=zip(model_paths,out_dirs);
#     writeJustTestScript(out_file_sh,val_data_path,iterations,batch_size,model_out_tups,face);


makeTestScript();
    

    



In [ ]:
def readErrEuclidean(log_file):
    log_data=util.readLinesFromFile(log_file);
    final_loss_str='minibatches processed: all'
    final_loss=[line_curr for line_curr in log_data if final_loss_str in line_curr];
    final_loss=final_loss[0].split(' ');
    final_loss=final_loss[-1].strip('"');
    final_loss=float(final_loss);
    return final_loss;

def makeAblationGraph():
#     dir_ablation=os.path.join(dir_server,'horse_project/ablation_studies');
#     dirs_type=['us','bl_ft','bl_tps'];
#     labels=['Ours','BL Finetune','BL TPS'];
#     colors=None;

    dir_ablation=os.path.join(dir_server,'horse_project/cvpr_figs/ablation_study_affine');
    dirs_type=['us_affine','bl_affine'];
    labels=['Ours Affine','BL Affine'];
    nums=range(500,3500,500)+[3531];
    colors=['b','g'];
    
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    iterations=2;
    batch_size=100;
    post_us=['_gt_pts.npy','_pred_pts.npy']

    avg_failures=[];
    avg_failures_kp=[];
    err_euc_all=[];
    for dir_curr in dirs_type:
        curr_failures=[];
        curr_failures_kp=[];
        curr_err_euc=[];
        for num_data in nums:
            print dir_curr,num_data;
            data_dir=os.path.join(dir_ablation,dir_curr,str(num_data))
#             ,'test_images');
            im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,data_dir,post_us,iterations,batch_size);
            errors_curr=us_getErrorsAll(val_data_path,data_dir,post_us,iterations,batch_size);
            failures,failures_kp=getErrRates(errors_curr,0.1);
            err_euc=readErrEuclidean(os.path.join(data_dir,'log_test.txt'));
            curr_err_euc.append(err_euc)
            curr_failures_kp.append(failures);
            curr_failures.append(failures_kp);
        err_euc_all.append(curr_err_euc);
        avg_failures.append(curr_failures);
        avg_failures_kp.append(curr_failures_kp);
#         break;
#     print avg_failures_kp[0][-1],avg_failures_kp[1][-1],avg_failures_kp[2][-1]
    
    out_file=os.path.join(dir_ablation,'comparison_kp_avg_failure.pdf');
    xAndYs=[(nums,vals_curr) for vals_curr in avg_failures];
    for val_curr in avg_failures:
        print val_curr[-1];
    print xAndYs
    xlabel='Training Data Size';
    ylabel='Failure %';
    
    visualize.plotSimple(xAndYs,out_file,xlabel=xlabel,ylabel=ylabel,legend_entries=labels);
    print out_file.replace(dir_server,click_str);
    
#     out_file=os.path.join(dir_ablation,'comparison_kp_avg_euc.png');
#     xAndYs=[(nums,vals_curr) for vals_curr in err_euc_all];
#     print xAndYs
#     xlabel='Training Data Size';
#     ylabel='Avg Normalized Euclidean Distance';
#     visualize.plotSimple(xAndYs,out_file,xlabel=xlabel,ylabel=ylabel,legend_entries=labels);
#     print out_file.replace(dir_server,click_str);
    
    
makeAblationGraph();

    
    

In [ ]:
def makeNeighborsHTML():
    horse_file='/home/SSD3/maheen-data/horse_project/neighbor_data/small_datasets/matches_5_2000_horse.txt';
    human_file='/home/SSD3/maheen-data/horse_project/neighbor_data/small_datasets/matches_5_2000_face.txt';
    dir_figures='/home/SSD3/maheen-data/horse_project/cvpr_figs';
    lines_horse=util.readLinesFromFile(horse_file);
    lines_human=util.readLinesFromFile(human_file);
    lines_horse=np.array(lines_horse);
    lines_human=np.array(lines_human);
    img_paths=[];
    captions=[];
    for line_horse in np.unique(lines_horse):
        idx_keep=lines_horse==line_horse;
#         print sum(idx_keep)
        lines_human_rel=lines_human[idx_keep];
        im_horse=line_horse[:line_horse.index(' ')];
        ims_match=[];
        for line_human in lines_human_rel:
            ims_match.append(line_human[:line_human.index(' ')]);
        files_curr=[im_horse]+ims_match;
#         print files_curr;
        files_curr=[util.getRelPath(file_curr,dir_server) for file_curr in files_curr]

        captions_curr=[' ']*len(files_curr);
        img_paths.append(files_curr);
        captions.append(captions_curr);
    out_file_html=os.path.join(dir_figures,'neighbors_viz.html');
    visualize.writeHTML(out_file_html,img_paths,captions);
    print out_file_html.replace(dir_server,click_str);
#         print line_horse
#         print lines_human_rel
#         break;

makeNeighborsHTML()
        

In [ ]:
def panicAttack():
    file_curr='/home/SSD3/maheen-data/horse_project/neighbor_data/small_datasets/matches_5_3531_horse_minloss.txt';
    lines=util.readLinesFromFile(file_curr);
    ims_train=[line_curr.split(' ')[0] for line_curr in lines];
    file_curr='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    lines=util.readLinesFromFile(file_curr);
    ims_test=[line_curr.split(' ')[0] for line_curr in lines];
    ims_train=np.array(ims_train);
    ims_test=np.array(ims_test);
    print np.sum(np.in1d(ims_test,ims_train));
    count=[file_curr for file_curr in ims_test if 'error_deleted' in file_curr]
    for file_curr in count:
        print file_curr.replace(dir_server,click_str);
    print len(count);
    print ims_test[12];
    print ims_test[0]
    print ims_train[0]


In [ ]:
def getBiggestDiff():
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    dir_figures='/home/SSD3/maheen-data/horse_project/cvpr_figs';
    iterations=2;
    batch_size=100;
    us_dir=os.path.join(dir_figures,'full_system_2loss_tps/test_images');
    
    them_dir=os.path.join(dir_figures,'baseline_finetune/test_images');
    out_file_html=os.path.join(dir_figures,'big_diff_bl_ft.html');
    
    them_dir=os.path.join(dir_figures,'baseline_tps/test_images');
    out_file_html=os.path.join(dir_figures,'big_diff_blTPS_ft.html');
    
#     val_data_path='/home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_test_allKP_minloss.txt';
#     iterations=2;
#     batch_size=50;
    
#     us_dir=os.path.join(dir_figures,'full_system_2loss_tps_sheep/test_images');
    
#     them_dir=os.path.join(dir_figures,'baseline_finetune_sheep/test_images');
#     out_file_html=os.path.join(dir_figures,'big_diff_bl_ft_sheep.html');
#     post_ims_us=['_nokp.jpg','_org.jpg','_org_nokp.jpg','.jpg']
#     post_ims_them=['_org.jpg','_org_nokp.jpg'];

#     them_dir=os.path.join(dir_figures,'baseline_tps_sheep/test_images');
#     out_file_html=os.path.join(dir_figures,'big_diff_blTPS_ft_sheep.html');
    post_ims_us=['_nokp.jpg','_org.jpg','_org_nokp.jpg','.jpg']
    post_ims_them=['_org.jpg','.jpg','_nokp.jpg'];
    
    
    post_us=['_gt_pts.npy','_pred_pts.npy']
    avgs=[];
    im_paths_all=[];
    
    for dir_curr in [us_dir,them_dir]:
        im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
        errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
        _,failures_kp=getErrRates(errors_curr,0.1);
        print failures_kp
        
        err=np.array(errors_curr);
        bin_keep=err>=0;
        err[err<0]=0;
        div=np.sum(bin_keep,1);
        sum_val=np.sum(err,1).astype(np.float);
        avg=sum_val/div;
#         print avg.shape;
#         print len(im_paths);
        avgs.append(avg);
#         print avg.shape
        im_paths_all.append(gt_pt_files);
    
    biggest_diff=avgs[1]-avgs[0];
    idx_sort=np.argsort(biggest_diff)[::-1];
    ims=[];
    captions=[];
    for idx_curr in idx_sort:
        file_curr=im_paths_all[0][idx_curr];
        file_curr=os.path.split(file_curr)[1];
        file_curr=file_curr[:file_curr.index('_gt')];
        files_us=[os.path.join(us_dir,file_curr+post_im_curr) for post_im_curr in post_ims_us ];
        captions_us=['us '+os.path.split(file_curr_curr)[1] for file_curr_curr in files_us];
        
        files_them=[os.path.join(them_dir,file_curr+post_im_curr) for post_im_curr in post_ims_them ];
        captions_them=['them '+os.path.split(file_curr_curr)[1] for file_curr_curr in files_them];
        files_all=files_us+files_them;
        captions_all=captions_us+captions_them;
        files_all=[util.getRelPath(file_curr,dir_server) for file_curr in files_all];
        
        ims.append(files_all);
        captions.append(captions_all);
    

    visualize.writeHTML(out_file_html,ims,captions);
    print out_file_html.replace(dir_server,click_str);
    
#     print biggest_diff.shape
#     biggest_diff_idx=np.argmax(biggest_diff);
#     print biggest_diff_idx;
#     print biggest_diff[0];
    
#     print im_paths_all[1][biggest_diff_idx].replace(dir_server,click_str);
#     print im_paths_all[0][biggest_diff_idx].replace(dir_server,click_str);
    
    
    
getBiggestDiff()

In [ ]:
def temp():
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    iterations=2;
    batch_size=100;
    post_us=['_gt_pts.npy','_pred_pts.npy']
    dir_curr='/home/SSD3/maheen-data/horse_project/cvpr_figs/gt_warp/test_images/test_images';
    im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
    errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
    failures,failures_kp=getErrRates(errors_curr,0.1);
    print failures_kp;

    dir_curr='/home/SSD3/maheen-data/horse_project/cvpr_figs/full_system_2loss_tps/test_images';
    im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
    errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
    failures,failures_kp=getErrRates(errors_curr,0.1);
    print failures_kp;
    
#     dir_curr='/home/SSD3/maheen-data/horse_project/ablation_studies_noDuplicates/us/3531/test_images';
#     im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
#     errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
#     failures,failures_kp=getErrRates(errors_curr,0.1);
#     print failures_kp;

#     dir_curr='/home/SSD3/maheen-data/horse_project/cvpr_figs_noDuplicates/gt_warp/test_images/test_images';
#     im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
#     errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
#     failures,failures_kp=getErrRates(errors_curr,0.1);
#     print failures_kp;

    dir_curr='/home/SSD3/maheen-data/horse_project/cvpr_figs/baseline_tps/test_images';
    im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
    errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
    failures,failures_kp=getErrRates(errors_curr,0.1);
    print failures_kp;
    
    dir_curr='/home/SSD3/maheen-data/horse_project/cvpr_figs/baseline_finetune/test_images';
    im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
    errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
    failures,failures_kp=getErrRates(errors_curr,0.1);
    print failures_kp;
    
    dir_curr='/home/SSD3/maheen-data/horse_project/cvpr_figs/random_neighbors';
    im_paths,gt_pt_files,pred_pt_files=us_getFilePres(val_data_path,dir_curr,post_us,iterations,batch_size);
    errors_curr=us_getErrorsAll(val_data_path,dir_curr,post_us,iterations,batch_size);
    failures,failures_kp=getErrRates(errors_curr,0.1);
    print failures_kp;

temp();

In [ ]:
import cv2
def saveImWithAnno(im_path,anno,out_path):
#     print idx;
    im=cv2.imread(im_path,1);
    # im=cv2.res tiize(im,(224,224));
    label=anno;
    x=label[:,0];
    y=label[:,1];
    color=(0,0,255);
#     colors=[(0,0,255),(0,255,0),(255,0,0),(255,255,0),(0,255,255)]
    colors=[(0,255,0)]*len(x)

    for label_idx in range(len(x)):
#         if label[label_idx,2]>0:
        cv2.circle(im,(x[label_idx],y[label_idx]),6,colors[label_idx],-1);
    cv2.imwrite(out_path,im);
    
    
def saveHTML():
    dir_sheep_results=os.path.join(dir_server,'horse_project/sheep_baseline_results');
    dir_input_data='/home/SSD3/maheen-data/horse_project/files_for_sheepCode'
    util.mkdir(dir_sheep_results);
    
    us_test='sheep_test_us_sheep_minloss.txt'
#     ,'horse_test_us_horse_minloss.txt'];
    us_test=os.path.join(dir_input_data,us_test)
#     for file_curr in us_test]
    
    them_test=['sheep_test_new.txt']
    # ,'horse_test.txt'];
    
    them_append='_TIF_result.txt'

    out_us=os.path.join(dir_sheep_results,'sheep_us/test_images')
    batch_size=50;
    num_iter=2;
    post_us=['_gt_pts.npy','_pred_pts.npy']
    avgs=[];
    im_paths_all=[];

    for dir_curr in [out_us]:
        im_paths,gt_pt_files,pred_pt_files=us_getFilePres(us_test,out_us,post_us,num_iter,batch_size);
        errors_curr=us_getErrorsAll(us_test,dir_curr,post_us,num_iter,batch_size);
        err=np.array(errors_curr);
        bin_keep=err>=0;
        err[err<0]=0;
        div=np.sum(bin_keep,1);
        sum_val=np.sum(err,1).astype(np.float);
        avg=sum_val/div;
        avgs.append(avg);
        im_paths_all.append(gt_pt_files);
    
    
    dir_sheep_results=os.path.join(dir_server,'horse_project/sheep_baseline_results');
    out_dir_im=os.path.join(dir_sheep_results,'sheep_tif_im_5p_new_correct');
    util.mkdir(out_dir_im);
    out_file_html=os.path.join(out_dir_im,'comparison.html')
    dir_input_data='/home/SSD3/maheen-data/horse_project/files_for_sheepCode'
    util.mkdir(dir_sheep_results);

    us_test='sheep_test_us_sheep_minloss.txt'
    # ,'horse_test_us_horse_minloss.txt'];
    # us_test=[os.path.join(dir_input_data,file_curr) for file_curr in us_test]
    out_us_org=out_us;
    out_us=os.path.join(dir_sheep_results,'sheep_us/test_images')
    
    batch_size=50;
    num_iter=2;

    them_test=[os.path.join(dir_input_data,file_curr) for file_curr in them_test];
    out_them=[file_curr[:file_curr.rindex('.')]+them_append for file_curr in them_test];
    gt_file_them=them_test[0];
    pred_file_them=out_them[0];
    them_errors_all=them_getErrorsAll(gt_file_them,pred_file_them);
    err=np.array(them_errors_all).astype(np.float);
    bin_keep=err>=0;
    err[err<0]=0;
    div=np.sum(bin_keep,1);
    sum_val=np.sum(err,1).astype(np.float);
    avg=sum_val/div;
    avgs.append(avg);
    
    post_ims_us=['_org.jpg','_org_nokp.jpg','.jpg']
    post_ims_them=['_org.jpg'];
    
    us_dir=out_us_org;
    them_dir=out_dir_im;
    biggest_diff=avgs[1]-avgs[0];
    idx_sort=np.argsort(biggest_diff)[::-1];
    print biggest_diff
    ims=[];
    captions=[];
    for idx_curr in idx_sort:
        file_curr=im_paths_all[0][idx_curr];
        file_curr=os.path.split(file_curr)[1];
        file_curr=file_curr[:file_curr.index('_gt')];
        files_us=[os.path.join(us_dir,file_curr+post_im_curr) for post_im_curr in post_ims_us ];
        captions_us=['us']*len(files_us);
        
        files_them=[os.path.join(them_dir,file_curr+post_im_curr) for post_im_curr in post_ims_them ];
        captions_them=['them']*len(files_them);
        files_all=files_us+files_them;
        captions_all=captions_us+captions_them;
        files_all=[util.getRelPath(file_curr,dir_server) for file_curr in files_all];
        
        ims.append(files_all);
        captions.append(captions_all);
    
    visualize.writeHTML(out_file_html,ims,captions);
    print out_file_html.replace(dir_server,click_str);

saveHTML()

In [ ]:
def makeSheepPeopleResultViz():
    dir_sheep_results=os.path.join(dir_server,'horse_project/sheep_baseline_results');
#     out_dir_im=os.path.join(dir_sheep_results,'sheep_tif_im_8p');
    out_dir_im=os.path.join(dir_sheep_results,'sheep_tif_im_5p_new_correct');
    util.mkdir(out_dir_im);
#     pre='';
    dir_input_data='/home/SSD3/maheen-data/horse_project/files_for_sheepCode'
    util.mkdir(dir_sheep_results);

    us_test='sheep_test_us_sheep_minloss.txt'
    out_us=os.path.join(dir_sheep_results,'sheep_us/test_images')
    batch_size=50;
    num_iter=2;

#     them_test=['sheep_test_copyForSupp.txt']
    them_test=['sheep_test_new.txt']
    them_test=[os.path.join(dir_input_data,file_curr) for file_curr in them_test];
    out_them=[file_curr[:file_curr.rindex('.')]+'_TIF_result.txt' for file_curr in them_test];

    gt_file_them=them_test[0];
    pred_file_them=out_them[0];

    # them_errors_all=them_getErrorsAll(gt_file_them,pred_file_them);
    im_paths,im_sizes,annos_gt=readGTFile(gt_file_them);
    annos_pred=readPredFile(pred_file_them);
    # print im_paths[0]
    # print annos_gt[0]
    # print annos_pred[0]
#     annos_gt=annos_pred;
    for i,(im_path,anno_gt,anno_pred) in enumerate(zip(im_paths,annos_gt,annos_pred)):
        if i<49:
            pre='1_';
        else:
            pre='2_'
            i=i-50;
        out_path=os.path.join(out_dir_im,pre+str(i+1)+'.png');
        saveImWithAnno(im_path,anno_gt,out_path)

        out_path=os.path.join(out_dir_im,pre+str(i+1)+'_org.jpg');
        saveImWithAnno(im_path,anno_pred,out_path)
        print out_path

# visualize.writeHTMLForFolder('/home/SSD3/maheen-data/horse_project/sheep_baseline_results/sheep_tif_im_8p');
makeSheepPeopleResultViz()

In [ ]:
def writeTestScriptAffine():
    dir_figures='/home/SSD3/maheen-data/horse_project/cvpr_figs';
    util.mkdir(dir_figures);
    out_dir_meta=os.path.join(dir_figures,'ablation_study_affine');
    out_file_sh=os.path.join(dir_figures,'commands_test_1.sh');
    file_th='/home/maheenrashid/Downloads/horses/torch/justTest.th';
    val_data_path='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_val_allKP_minLoss_clean.txt';
    
    util.mkdir(out_dir_meta);
    
    iterations=2;
    batch_size=100;
    face=False;
    model_out_tups=[];
    
    range_vals=range(500,3500,500);
    range_vals=range_vals+[3531];
    
    model_all_tups=[];
        
#     out_dir_bl=os.path.join(out_dir_meta,'bl_affine');
#     util.mkdir(out_dir_bl);
#     dirs_bl=['horse_project/face_baselines_small_data_affine_200_1e-3_100/matches_5_500',
#             'horse_project/face_baselines_small_data_affine_200_1e-3_100/matches_5_1000',
#             'horse_project/face_baselines_small_data_affine_200_1e-3_100/matches_5_1500',
#             'horse_project/face_baselines_small_data_affine_200_1e-3_100/matches_5_2000/resume',
#             'horse_project/face_baselines_small_data_affine_200_1e-3_100/matches_5_2500/resume',
#             'horse_project/face_baselines_small_data_affine_200_1e-3_100/matches_5_3000',
#             'horse_project/ext_disk/face_baselines_small_data/matches_5_3531_horse_minloss_affine']
    
#     models_bl=[os.path.join(dir_server,dir_curr,'final/model_all_final.dat') for dir_curr in dirs_bl];
#     out_dirs=[os.path.join(out_dir_bl,str(num_curr)) for num_curr in range_vals];
#     tups=zip(models_bl,out_dirs);
#     model_all_tups.extend(tups);
    
    out_dir_bl=os.path.join(out_dir_meta,'us_affine');
    util.mkdir(out_dir_bl);
    dirs_bl=['horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_500_horse_minloss',
            'horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_1000_horse_minloss',
            'horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_1500_horse_minloss',
            'horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_2000_horse_minloss/resume',
            'horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_2500_horse_minloss',
            'horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_3000_horse_minloss/resume',
            'horse_project/full_system_small_data_eye_1e-2_10_100_affine/matches_5_3531_horse_minloss']
    models_bl=[os.path.join(dir_server,dir_curr,'final/model_all_final.dat') for dir_curr in dirs_bl];
    out_dirs=[os.path.join(out_dir_bl,str(num_curr)) for num_curr in range_vals];
    tups=zip(models_bl,out_dirs);
    model_all_tups.extend(tups);
    
    face=False;
    commands_all=[];
    for model_path_curr,out_dir_curr in model_all_tups:
        command_curr=['th',file_th];
        command_curr=command_curr+['-val_data_path',val_data_path];
        command_curr=command_curr+['-iterations',str(iterations)];
        command_curr=command_curr+['-batchSize',str(batch_size)];
        command_curr=command_curr+['-full_model_path',model_path_curr];
        command_curr=command_curr+['-outDirTest',out_dir_curr];
        if face:
            command_curr=command_curr+['-face'];
        command_curr=' '.join(command_curr);
        print command_curr;
        commands_all.append(command_curr);
    
    util.writeFile(out_file_sh,commands_all);
    print out_file_sh;

writeTestScriptAffine();